# Goblin Freeze & Thaw Demo

Launch a small goblin (pipe-only worker), freeze it, and thaw clones using `pdum.criu.goblins`. The notebook now runs the doctor diagnostics up front so you immediately know if CRIU tooling is ready.


## Doctor

Run the same diagnostics as `pdum-criu doctor` so the live demo only executes on a ready system.


In [1]:

import sys
from rich.console import Console
from pdum.criu import utils

console = Console()
DOCTOR_RESULTS: list[tuple[str, bool, str | None]] = []

if not sys.platform.startswith("linux"):
    LIVE_READY = False
    LIVE_REASON = f"CRIU requires Linux (detected {sys.platform})."
    console.print(f"[bold red]{LIVE_REASON}[/]")
else:
    DOCTOR_RESULTS = utils.doctor_check_results(verbose=False)
    LIVE_READY = all(ok for _, ok, _ in DOCTOR_RESULTS)
    for label, ok, message in DOCTOR_RESULTS:
        status = "[bold green]OK[/]" if ok else "[bold red]FAIL[/]"
        console.print(f"{status} {label}")
        if message:
            console.print(f"    {message}")
    if LIVE_READY:
        console.print("[bold green]Doctor checks passed; continuing.[/]")
    else:
        console.print("[bold yellow]Doctor reported issues; later cells will skip live work.[/]")
    failures = [f"{label}: {message or 'see CLI output'}" for label, ok, message in DOCTOR_RESULTS if not ok]
    LIVE_REASON = "Doctor checks passed" if not failures else "Doctor checks failed: " + " | ".join(failures)


OK Password-less sudo

OK CRIU

OK CRIU-ns

OK pgrep

OK sudo closefrom_override

Doctor checks passed; continuing.

## Environment Check & Helpers

In [2]:

from __future__ import annotations

import asyncio
import os
import signal
import subprocess
import textwrap
import time
from pathlib import Path

from pdum.criu import goblins, utils

BASE_DIR = Path("/tmp/_goblin_runs")
BASE_DIR.mkdir(parents=True, exist_ok=True)
RUN_DIRS: list[Path] = []
SERVERS = []
FREEZE_DIR: Path | None = None

COUNTER_MARKER = "PDUM_GOBLIN_COUNTER"

COUNTER_SCRIPT = textwrap.dedent(
    f"""
    # {COUNTER_MARKER}
    import os
    import sys

    os.setsid()
    counter = 0
    print("ready", flush=True)

    for line in sys.stdin:
        if not line:
            break
        counter += 1
        sys.stdout.write(f"{{counter}}:{{line}}")
        sys.stdout.flush()
    """
)


def fresh_run_dir(label: str) -> Path:
    path = BASE_DIR / f"{label}-{int(time.time() * 1000)}"
    path.mkdir(parents=True, exist_ok=True)
    RUN_DIRS.append(path)
    return path


def ensure_user_ownership(path: Path) -> None:
    subprocess.run(
        [
            "sudo",
            "chown",
            "-R",
            f"{os.getuid()}:{os.getgid()}",
            str(path),
        ],
        check=False,
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
    )


def remove_dir(path: Path) -> None:
    subprocess.run(["sudo", "rm", "-rf", str(path)], check=False)


In [3]:
class GoblinServer:
    def __init__(self, run_dir: Path):
        self.proc = subprocess.Popen(
            [os.environ.get("PYTHON", "python3"), "-u", "-c", COUNTER_SCRIPT],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
        )
        SERVERS.append(self.proc)
        self.proc.stdout.readline()

    def stop(self) -> None:
        if self.proc.poll() is None:
            self.proc.terminate()
            try:
                self.proc.wait(timeout=3)
            except subprocess.TimeoutExpired:
                self.proc.kill()


class GoblinClient:
    def __init__(self, stdin, stdout):
        self.stdin = stdin
        self.stdout = stdout

    def send(self, text: str) -> str:
        payload = text.rstrip("\n") + "\n"
        self.stdin.write(payload.encode("utf-8"))
        self.stdin.flush()
        return self.stdout.readline().decode("utf-8").strip()


In [4]:

if LIVE_READY:
    run_dir = fresh_run_dir("server")
    server = GoblinServer(run_dir)
    client = GoblinClient(server.proc.stdin, server.proc.stdout)
    print(client.send("alpha"))
    print(client.send("beta"))
    located_pid = utils.psgrep(COUNTER_MARKER)
    print(f"psgrep located PID {located_pid} (server PID: {server.proc.pid})")
else:
    server = None
    print(f"Skipping launch: {LIVE_REASON}")


1:alpha
2:beta
psgrep located PID 352949 (server PID: 352949)


In [ ]:
if LIVE_READY and server is not None:
    FREEZE_DIR = fresh_run_dir("freeze")
    log_path = goblins.freeze(server.proc.pid, FREEZE_DIR, leave_running=False)
    ensure_user_ownership(FREEZE_DIR)
    print(f"Frozen goblin into {FREEZE_DIR}\nLog: {log_path}")
    server.stop()
else:
    print(f"Skipping freeze: {LIVE_REASON}")


Frozen goblin into /tmp/_goblin_runs/freeze-1762548393086
Log: /tmp/_goblin_runs/freeze-1762548393086/goblin-freeze.352949.log


In [ ]:
# if LIVE_READY and FREEZE_DIR:
#     thawed = goblins.thaw(FREEZE_DIR)
#     clone_client = GoblinClient(thawed.stdin, thawed.stdout)
#     print(clone_client.send("gamma"))
#     print(clone_client.send("delta"))
#     os.kill(thawed.pid, signal.SIGTERM)
#     thawed.stdin.close()
#     thawed.stdout.close()
#     thawed.stderr.close()
# else:
#     print(f"Skipping sync thaw: {LIVE_REASON}")


Traceback (most recent call last):
  File "/usr/sbin/criu-ns", line 333, in <module>
    res = wrap_restore()
          ^^^^^^^^^^^^^^
  File "/usr/sbin/criu-ns", line 214, in wrap_restore
    return _wait_for_process_status(criu_pid)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/sbin/criu-ns", line 72, in _wait_for_process_status
    (pid, status) = os.wait()
                    ^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# if LIVE_READY and FREEZE_DIR:
#     thawed = goblins.thaw(FREEZE_DIR)
#     clone_client = GoblinClient(thawed.stdin, thawed.stdout)
#     print(clone_client.send("gamma"))
#     print(clone_client.send("delta"))
#     os.kill(thawed.pid, signal.SIGTERM)
#     thawed.stdin.close()
#     thawed.stdout.close()
#     thawed.stderr.close()
# else:
#     print(f"Skipping sync thaw: {LIVE_REASON}")


In [ ]:
for proc in SERVERS:
    if proc.poll() is None:
        proc.terminate()
        try:
            proc.wait(timeout=3)
        except subprocess.TimeoutExpired:
            proc.kill()

for path in RUN_DIRS:
    remove_dir(path)

print("Cleaned up goblin runs.")
